<a href="https://colab.research.google.com/github/urnotirisfeng/AI/blob/main/TP/TP_Recherche_Arborescente_Informee%20_Correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP Intelligence Artificielle - Recherche Arborescente Informée

# **Partie 0 : Visualisation d'états, Classe Taquin et Classe Node**

Nous récupérons les classes du TP précédent ainsi que la fonction visualise_state.

In [3]:
from IPython.display import display, HTML

def visualize_state(state):
    """
    使用 HTML 可视化 Taquin 拼图的当前状态。
    空白格（值为 0）显示为灰色，其它数字方块显示为浅蓝色。
    """
    html = "<table>"
    for row in state:
        html += "<tr>"
        for tile in row:
            if tile == 0:
                # 空白格（0）显示为灰色，无数字
                html += "<td style='background-color: lightgray; width: 30px; height: 30px; text-align: center; font-size: 20px;'> </td>"
            else:
                # 有数字的格子显示为浅蓝色
                html += f"<td style='background-color: lightblue; width: 30px; height: 30px; text-align: center; font-size: 20px;'>{tile}</td>"
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

class Taquin:
    """
    定义 Taquin 拼图问题。
    包含初始状态、目标状态、拼图大小，以及问题的基本操作。
    """

    def __init__(self, initial_state, goal_state, size):
        self.initial_state = initial_state  # 初始拼图状态
        self.goal_state = goal_state        # 目标拼图状态
        self.size = size                    # 拼图维度（如 3 表示 3x3）

    def actions(self, state):
        """
        给定当前状态，返回可行的操作（上下左右移动空白格）。
        """
        # 找到空白格（值为 0）的位置
        row, col = next(
            (r, c)
            for r, row in enumerate(state)
            for c, val in enumerate(row)
            if val == 0
        )

        # 判断哪些方向可以移动
        possible_actions = []
        if row > 0:
            possible_actions.append("up")
        if row < self.size - 1:
            possible_actions.append("down")
        if col > 0:
            possible_actions.append("left")
        if col < self.size - 1:
            possible_actions.append("right")

        return possible_actions

    def result(self, state, action):
        """
        返回在当前状态下执行某个动作后的新状态（二维列表）。
        不改变原始状态，而是返回其副本。
        """
        # 创建状态的深拷贝
        new_state = [list(row) for row in state]

        # 找到空白格位置
        row, col = next(
            (r, c)
            for r, row in enumerate(state)
            for c, val in enumerate(row)
            if val == 0
        )

        # 根据动作交换空白格与相邻方块
        if action == "up":
            new_state[row][col], new_state[row - 1][col] = new_state[row - 1][col], new_state[row][col]
        elif action == "down":
            new_state[row][col], new_state[row + 1][col] = new_state[row + 1][col], new_state[row][col]
        elif action == "left":
            new_state[row][col], new_state[row][col - 1] = new_state[row][col - 1], new_state[row][col]
        elif action == "right":
            new_state[row][col], new_state[row][col + 1] = new_state[row][col + 1], new_state[row][col]

        return new_state

    def is_goal(self, state):
        """判断当前状态是否是目标状态。"""
        return state == self.goal_state

    def cost(self, state, action):
        """定义执行某个动作的代价。这里默认每一步代价为1。"""
        return 1

class Node:
    """
    搜索树中的节点。
    每个节点记录当前状态、父节点、采取的动作、路径代价以及深度信息。
    """

    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state              # 当前状态
        self.parent = parent            # 父节点（前一个状态）
        self.action = action            # 从父节点到当前节点采取的动作
        self.path_cost = path_cost      # 从初始状态到此节点的总代价
        self.depth = 0 if parent is None else parent.depth + 1  # 深度

    def expand(self, problem):
        """
        生成所有可达的子节点（下一步状态）。
        返回一个包含子节点的列表。
        """
        return [
            self.child_node(problem, action)
            for action in problem.actions(self.state)
        ]

    def child_node(self, problem, action):
        """
        对某个动作，生成并返回对应的子节点。
        """
        next_state = problem.result(self.state, action)  # 应用动作后的状态
        next_node = Node(
            next_state,
            parent=self,
            action=action,
            path_cost=self.path_cost + problem.cost(self.state, action)
        )
        return next_node

    def solution(self):
        """
        返回从初始节点到当前节点的动作序列。
        """
        return [node.action for node in self.path()[1:]]  # 去掉根节点的 None 动作

    def path(self):
        """
        返回从根节点到当前节点的路径（节点列表）。
        可用于重建解决方案路径。
        """
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))  # 从根到当前节点

# **Partie 1 : Heuristiques pour le jeu de Taquin**

## **1.1 H$_1$ : Nombre de tuiles mal placées**

### **Exercice 1**

Écrivez une fonction qui calcule le nombre de tuiles mal placées dans un état du jeu de Taquin

In [4]:
def misplaced_tiles(state, goal_state):
    """
    计算当前状态中有多少块拼图位置不正确（不在目标位置）。
    常用于启发式搜索（如 A*）中的估价函数。

    参数:
        state: 当前拼图状态（二维列表）
        goal_state: 目标拼图状态（二维列表）

    返回:
        int: 拼图中位置错误的方块数量（不包括空白块 0）
    """

    misplaced_count = 0  # 初始化计数器，记录错位块的数量

    # 遍历拼图中每个位置
    for i in range(len(state)):           # 遍历行
        for j in range(len(state[0])):    # 遍历列
            # 如果当前方块不为空（0），且与目标状态不同，则视为错位
            if state[i][j] != 0 and state[i][j] != goal_state[i][j]:
                misplaced_count += 1

    return misplaced_count  # 返回总的错位块数


### **Exercice 2**

Calculez le nombre de tuiles mal placées pour les états suivants :
1. [[1, 2, 3], [4, 5, 0], [6, 7, 8]]
2. [[1, 2, 3], [0, 5, 6], [4, 7, 8]]    
3. [[1, 0, 3], [4, 2, 5], [7, 8, 6]]
4. [[1, 0, 3], [4, 5, 2], [7, 6, 8]]
5. [[1, 0, 3], [4, 2, 5], [6, 7, 8]]
6. [[1, 2, 3], [4, 0, 6], [7, 5, 8]]
7. [[1, 2, 3], [0, 4, 6], [7, 5, 8]]
8. [[1, 2, 3], [4, 6, 0], [7, 5, 8]]
9. [[1, 3, 6], [4, 2, 5], [7, 0, 8]]  
10. [[1, 3, 6], [4, 2, 0], [7, 5, 8]]  
11. [[1, 3, 6], [4, 0, 2], [7, 5, 8]]  
12. [[3, 1, 2], [4, 6, 5], [7, 0, 8]]  
13. [[8, 1, 2], [0, 4, 3], [7, 6, 5]]
14. [[1, 4, 2], [7, 0, 6], [5, 3, 8]]
15. [[2, 8, 3], [1, 6, 4], [7, 0, 5]]

In [5]:
# 定义一组初始状态（列表中的每个 3x3 二维数组表示一个拼图配置）
initial_states = [
    [[1, 2, 3], [4, 5, 0], [6, 7, 8]],
    [[1, 2, 3], [0, 5, 6], [4, 7, 8]],
    [[1, 0, 3], [4, 2, 5], [7, 8, 6]],
    [[1, 0, 3], [4, 5, 2], [7, 6, 8]],
    [[1, 0, 3], [4, 2, 5], [6, 7, 8]],
    [[1, 2, 3], [4, 0, 6], [7, 5, 8]],
    [[1, 2, 3], [0, 4, 6], [7, 5, 8]],
    [[1, 2, 3], [4, 6, 0], [7, 5, 8]],
    [[1, 3, 6], [4, 2, 5], [7, 0, 8]],
    [[1, 3, 6], [4, 2, 0], [7, 5, 8]],
    [[1, 3, 6], [4, 0, 2], [7, 5, 8]],
    [[3, 1, 2], [4, 6, 5], [7, 0, 8]],
    [[8, 1, 2], [0, 4, 3], [7, 6, 5]],
    [[1, 4, 2], [7, 0, 6], [5, 3, 8]],
    [[2, 8, 3], [1, 6, 4], [7, 0, 5]]
]

# 定义目标状态（拼图完成的状态）
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

# 遍历每一个初始状态，逐个评估
for state in initial_states:
    print('Checking state')  # 打印提示信息

    visualize_state(state)  # 使用 HTML 表格显示拼图当前状态（需要在 Jupyter 环境中运行）

    # 使用前面定义的启发式函数计算当前状态的错位方块数
    print('The state has', misplaced_tiles(state, goal_state), 'misplaced tiles')

    print()  # 打印空行用于分隔每个输出块

Checking state


1,2,3
4,5,
6,7,8


The state has 3 misplaced tiles

Checking state


1,2,3
,5,6
4,7,8


The state has 3 misplaced tiles

Checking state


...

The state has 7 misplaced tiles

Checking state


1,4,2
7,,6
5,3,8


The state has 6 misplaced tiles

Checking state


2,8,3
1,6,4
7,,5


The state has 6 misplaced tiles



## **1.2 H$_2$ : Distance de Manhattan**

### **Exercice 3**
Écrivez une fonction qui calcule la distance de Manhattan entre un état et l'état but. La fonction **next()** pourrait être utile pour trouver la position d'une tuile donnée dans l'état but.

In [8]:
def manhattan_distance(state, goal_state):
    """
    计算当前拼图状态到目标状态的曼哈顿距离。
    对每个非空白块，计算它与目标位置之间的水平 + 垂直距离之和。

    参数:
        state: 当前状态（3x3 二维数组）
        goal_state: 目标状态（3x3 二维数组）

    返回:
        int: 曼哈顿距离的总和，表示该状态离目标状态的“估计代价”
    """

    distance = 0  # 初始化总曼哈顿距离

    # 遍历拼图中的每个格子
    for i in range(len(state)):          # 遍历每一行
        for j in range(len(state[0])):   # 遍历每一列
            if state[i][j] != 0:  # 忽略空白格（0）

                # 查找当前 tile 在目标状态中的位置
                goal_row, goal_col = next(
                    (r, c)                         # r = 行号, c = 列号
                    for r, row in enumerate(goal_state)      # 遍历目标状态每一行
                    for c, val in enumerate(row)             # 遍历该行中的每个元素
                    if val == state[i][j]        # 如果目标状态中找到相同 tile
                )

                # 计算当前位置 (i, j) 到目标位置 (goal_row, goal_col) 的曼哈顿距离
                distance += abs(i - goal_row) + abs(j - goal_col)

    return distance  # 返回总距离

### **Exercice 4**
1. Calculez la distance de Manhattan des états de l'exercice 2.
2. Que pouvez-vous observer par rapport aux deux heuristiques ?

In [9]:
# 定义多个拼图初始状态（3x3 的拼图，每个列表代表一个状态）
# 每个状态都是一个列表的列表（3 行，每行 3 列）
initial_states = [
    [[1, 2, 3], [4, 5, 0], [6, 7, 8]],   # 几乎完成，只差一个 tile
    [[1, 2, 3], [0, 5, 6], [4, 7, 8]],
    [[1, 0, 3], [4, 2, 5], [7, 8, 6]],
    [[1, 0, 3], [4, 5, 2], [7, 6, 8]],
    [[1, 0, 3], [4, 2, 5], [6, 7, 8]],
    [[1, 2, 3], [4, 0, 6], [7, 5, 8]],
    [[1, 2, 3], [0, 4, 6], [7, 5, 8]],
    [[1, 2, 3], [4, 6, 0], [7, 5, 8]],
    [[1, 3, 6], [4, 2, 5], [7, 0, 8]],
    [[1, 3, 6], [4, 2, 0], [7, 5, 8]],
    [[1, 3, 6], [4, 0, 2], [7, 5, 8]],
    [[3, 1, 2], [4, 6, 5], [7, 0, 8]],
    [[8, 1, 2], [0, 4, 3], [7, 6, 5]],
    [[1, 4, 2], [7, 0, 6], [5, 3, 8]],
    [[2, 8, 3], [1, 6, 4], [7, 0, 5]]
]

# 定义目标状态（完成拼图后的正确配置）
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]


# 遍历每一个初始状态，逐个进行评估和可视化
for state in initial_states:
    print('Checking state')  # 显示当前正在处理的状态

    visualize_state(state)  # 使用 HTML 表格在 Jupyter Notebook 中直观显示当前拼图布局

    # 计算当前状态与目标状态之间的曼哈顿距离（估计最少需要几步到达目标）
    print('The state has a Manhattan distance to the goal state of',
          manhattan_distance(state, goal_state))

    # 计算当前状态中有多少块 tile 不在正确位置（不含空格）
    print('The state has', misplaced_tiles(state, goal_state), 'misplaced tiles')

    print()  # 打印空行用于分隔各个状态的结果输出

Checking state


1,2,3
4,5,
6,7,8


The state has a Manhattan distance to the goal state of 5
The state has 3 misplaced tiles

Checking state


1,2,3
,5,6
4,7,8


The state has a Manhattan distance to the goal state of 3
The state has 3 misplaced tiles

Checking state


...

The state has a Manhattan distance to the goal state of 11
The state has 7 misplaced tiles

Checking state


1,4,2
7,,6
5,3,8


The state has a Manhattan distance to the goal state of 10
The state has 6 misplaced tiles

Checking state


2,8,3
1,6,4
7,,5


The state has a Manhattan distance to the goal state of 9
The state has 6 misplaced tiles



# **Partie 2 : Algorithme A étoile**

### **Exercice 5**

Créer une fonction A_etoile qui prend un objet problem en entrée ainsi qu'une heuristique et exécute l'algorithme A.*

Votre fonction doit :

1. Retourner le nœud objectif trouvé ainsi que le nombre de nœuds explorés.
2. Retourner None si l'algorithme explore tout l'arbre sans trouver le nœud objectif.
3. Prendre en compte un budget d'exploration pour arrêter l'algorithme si aucune solution n'est trouvée après plusieurs itérations (utile pour les grands problèmes). Le budget sera également un paramètre d'entrée de la fonction, avec float('inf') comme valeur par défaut.

Pour la **frontière** et l'ensemble des **nœuds déjà explorés**, vous pouvez utiliser des listes. Cependant, la frontière doit contenir les nœuds à explorer ainsi que leurs coûts (coût réel + heuristique). Pour trouver l'élément (nœud, coût) de coût minimal dans la frontière, effectuez une boucle **for** pour trouver l'indice, puis utilisez pop.

Pour ajouter un élément à une liste, vous pouvez utiliser **liste.append(element)**.

Rappelez-vous que la vérification **is_goal** doit être effectuée avant d'explorer les enfants.

In [10]:
def a_etoile(problem, heuristic, Max_it = float('inf')):
    """
    A*搜索算法的实现，用于在拼图问题等图搜索问题中寻找从初始状态到目标状态的最优路径。

    参数：
        problem: 一个定义好的问题对象（例如 Taquin 类实例），提供初始状态、动作、目标检测、状态转换等方法。
        heuristic: 启发式函数（例如 misplaced_tiles 或 manhattan_distance），用于估算当前状态到目标的距离。
        Max_it: 最大迭代次数（可选，用于限制搜索深度，默认为无穷）

    返回：
        (solution_node, iteration_count) 元组
            - solution_node: 到达目标状态的节点（包含路径、动作序列等）
            - iteration_count: 实际迭代次数（用于评估搜索开销）
        若未找到解决方案，则返回 (None, iteration_count)
    """

    # 初始化 frontier：优先队列（列表），元素为 (f值, 节点)
    frontier = [(0, Node(problem.initial_state))]

    # explored：用于记录已经访问过的状态，避免重复扩展
    explored = []

    it = 0  # 用于计数迭代次数（诊断搜索效率）

    # 主循环：只要还有待探索的节点，且未超过最大迭代次数
    while frontier and it < Max_it:
        it += 1  # 增加迭代计数器

        # 手动搜索 frontier 中 f 值最小的节点（因为这里用的是普通列表）
        min_f = float('inf')  # 当前最小的 f 值
        min_index = -1        # 最小 f 值对应的索引
        for i, (f, node) in enumerate(frontier):
            if f < min_f:
                min_f = f
                min_index = i

        # 将 f 值最小的节点从 frontier 中移除，作为当前要扩展的节点
        _, node = frontier.pop(min_index)

        # 如果该节点已经是目标状态，则返回结果
        if problem.is_goal(node.state):
            return node, it  # 成功找到解决方案，返回最终节点和迭代次数

        # 否则将该节点的状态加入 explored 集合，表示已访问
        explored.append(node.state)

        # 对当前节点进行扩展（生成所有合法子节点）
        for child in node.expand(problem):
            if child.state not in explored:
                # 计算代价函数 f(n) = g(n) + h(n)
                g = child.path_cost  # 实际路径代价（从起点到当前节点）
                h = heuristic(child.state, problem.goal_state)  # 启发式估值
                f = g + h  # 总估价函数

                # 将子节点加入 frontier 中等待后续探索
                frontier.append((f, child))

    # 如果退出循环，说明没有找到目标状态（或达到最大迭代次数）
    return None, it


### **Exercice 6**

Appliquez votre fonction A_etoile pour résoudre un Taquin 3x3 avec l'état initial [[1, 2, 3], [4, 5, 6], [0, 7, 8]] et les deux heuristiques.

Imprimez :
1. Le nombre de nœuds explorés par l'algorithme.
2. Le chemin permettant de passer de l'état initial à l'état objectif (la liste des actions effectuées).
3. Que pouvez-vous observer entre les deux heuristiques ?

In [14]:
# 定义初始状态和目标状态
initial_state = [[1, 2, 3], [4, 5, 0], [6, 7, 8]]
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

# 打印并可视化初始状态（需在 Jupyter Notebook 中运行）
print('Initial state')
visualize_state(initial_state)

# 创建 Taquin 问题实例（定义初始状态、目标状态和拼图大小）
problem = Taquin(initial_state, goal_state, len(initial_state[0]))

# 使用错位块（misplaced tiles）启发函数运行 A* 算法
solution_node_misplaced, it_misplaced = a_etoile(
    problem,
    misplaced_tiles,
    Max_it = 20000
)

# 使用曼哈顿距离（manhattan distance）启发函数运行 A* 算法
solution_node_manhattan, it_manhattan = a_etoile(
    problem,
    manhattan_distance,
    Max_it = 20000
)

# 打印使用第一个启发函数的结果（错位块数）
if solution_node_misplaced:
    print("✅ Solution with 1st heuristic (misplaced tiles) found in", it_misplaced, "iterations")
    print("🔁 Actions:", solution_node_misplaced.solution())

    # 如果需要，可取消注释以下代码来可视化整条路径
    # t = 1
    # for node in solution_node_misplaced.path():
    #     print(t, "-th element of the path")
    #     visualize_state(node.state)
    #     t += 1
else:
    print("❌ A* with h1 (misplaced tiles) did not find a solution.")

# 打印使用第二个启发函数的结果（曼哈顿距离）
if solution_node_manhattan:
    print("✅ Solution with 2nd heuristic (manhattan distance) found in", it_manhattan, "iterations")
    print("🔁 Actions:", solution_node_manhattan.solution())

    # 如果需要，可取消注释以下代码来可视化整条路径
    # t = 1
    # for node in solution_node_manhattan.path():
    #     print(t, "-th element of the path")
    #     visualize_state(node.state)
    #     t += 1
else:
    print("❌ A* with h2 (manhattan distance) did not find a solution.")


Initial state


1,2,3
4,5,
6,7,8


✅ Solution with 1st heuristic (misplaced tiles) found in 196 iterations
🔁 Actions: ['down', 'left', 'left', 'up', 'right', 'down', 'right', 'up', 'left', 'left', 'down', 'right', 'right']
✅ Solution with 2nd heuristic (manhattan distance) found in 85 iterations
🔁 Actions: ['down', 'left', 'left', 'up', 'right', 'down', 'right', 'up', 'left', 'left', 'down', 'right', 'right']


# **Partie 3 : Comparaison empirique de DFS, BFS, A etoile avec H1 et A etoile avec H2**

Nous allons comparer les quatre algorithmes sur les 15 instances du jeu de Taquin de l'exercice 2. Pour cela, voici les résultats de BFS et DFS sur ces 15 instances. Attention au format des tableaux. Les noms des colonnes peuvent différer de ceux que vous avez utilisés. De plus, un path_length égal à -1 signifie que l'algorithme n'a pas trouvé la solution avant d'atteindre le budget d'exploration.
我们将对练习 2 中的 15 个 Taquin 拼图实例使用四种算法进行比较。
为此，这里是 BFS 和 DFS 在这 15 个实例上的结果。
请注意表格的格式，列名可能与你所使用的不同。
此外，若 path_length 的值为 -1，表示该算法在达到探索预算之前未能找到解决方案。

In [ ]:
results_BFS = [
{'initial_state': [[1, 2, 3], [4, 5, 0], [6, 7, 8]],'algorithm': 'breadth_first_search', 'num_explorations': 1846, 'path_length': 13, 'execution_time': 0.2773609161376953},
{'initial_state': [[1, 2, 3], [0, 5, 6], [4, 7, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 6, 'path_length': 3, 'execution_time': 0.0},
{'initial_state': [[1, 0, 3], [4, 2, 5], [7, 8, 6]], 'algorithm': 'breadth_first_search', 'num_explorations': 7, 'path_length': 3, 'execution_time': 0.0},
{'initial_state': [[1, 0, 3], [4, 5, 2], [7, 6, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 10911, 'path_length': 17, 'execution_time': 8.975327491760254},
{'initial_state': [[1, 0, 3], [4, 2, 5], [6, 7, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 4960, 'path_length': 15, 'execution_time': 1.831636905670166},
{'initial_state': [[1, 2, 3], [4, 0, 6], [7, 5, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 3, 'path_length': 2, 'execution_time': 0.0},
{'initial_state': [[1, 2, 3], [0, 4, 6], [7, 5, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 8, 'path_length': 3, 'execution_time': 0.0},
{'initial_state': [[1, 2, 3], [4, 6, 0], [7, 5, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 8, 'path_length': 3, 'execution_time': 0.0},
{'initial_state': [[1, 3, 6], [4, 2, 5], [7, 0, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 36.05873966217041},
{'initial_state': [[1, 3, 6], [4, 2, 0], [7, 5, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 20, 'path_length': 5, 'execution_time': 0.0},
{'initial_state': [[1, 3, 6], [4, 0, 2], [7, 5, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 62, 'path_length': 6, 'execution_time': 0.0019948482513427734},
{'initial_state': [[3, 1, 2], [4, 6, 5], [7, 0, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 40.92391228675842},
{'initial_state': [[8, 1, 2], [0, 4, 3], [7, 6, 5]], 'algorithm': 'breadth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 40.67150044441223},
{'initial_state': [[1, 4, 2], [7, 0, 6], [5, 3, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 3321, 'path_length': 14, 'execution_time': 0.8201050758361816},
{'initial_state': [[2, 8, 3], [1, 6, 4], [7, 0, 5]], 'algorithm': 'breadth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 42.47766828536987}
]
results_DFS = [
{'initial_state': [[1, 2, 3], [4, 5, 0], [6, 7, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 69.51113247871399},
{'initial_state': [[1, 2, 3], [0, 5, 6], [4, 7, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 27, 'path_length': 27, 'execution_time': 0.0},
{'initial_state': [[1, 0, 3], [4, 2, 5], [7, 8, 6]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 60.40420460700989},
{'initial_state': [[1, 0, 3], [4, 5, 2], [7, 6, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 62.47419261932373},
{'initial_state': [[1, 0, 3], [4, 2, 5], [6, 7, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 67.59605073928833},
{'initial_state': [[1, 2, 3], [4, 0, 6], [7, 5, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 412, 'path_length': 406, 'execution_time': 0.02211737632751465},
{'initial_state': [[1, 2, 3], [0, 4, 6], [7, 5, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 65.1717848777771},
{'initial_state': [[1, 2, 3], [4, 6, 0], [7, 5, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 886, 'path_length': 871, 'execution_time': 0.08270859718322754},
{'initial_state': [[1, 3, 6], [4, 2, 5], [7, 0, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 66.5749568939209},
{'initial_state': [[1, 3, 6], [4, 2, 0], [7, 5, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 65.61762142181396},
{'initial_state': [[1, 3, 6], [4, 0, 2], [7, 5, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 66.86844420433044},
{'initial_state': [[3, 1, 2], [4, 6, 5], [7, 0, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 80.54781603813171},
{'initial_state': [[8, 1, 2], [0, 4, 3], [7, 6, 5]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 75.41818499565125},
{'initial_state': [[1, 4, 2], [7, 0, 6], [5, 3, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 69.51109766960144},
{'initial_state': [[2, 8, 3], [1, 6, 4], [7, 0, 5]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 83.47795271873474}
]

### **Exercice 7**
Utilisez (et adaptez si nécessaire) votre fonction compare_algorithms du premier TP pour résoudre tous les jeux avec les deux versions de l'algorithme A etoile.


In [17]:
import time  # 用于测量执行时间

def compare_algorithms(initial_states, goal_state, heuristics, max_it):
    """
    比较不同启发式函数的 A* 算法在多个 Taquin 初始状态上的表现。

    参数：
        initial_states: 一个列表，包含多个拼图初始状态（3x3 的二维数组）
        goal_state: 拼图目标状态（3x3）
        heuristics: 启发式函数列表，例如 [misplaced_tiles, manhattan_distance]
        max_it: A* 允许的最大迭代次数

    返回：
        results: 包含每个启发函数对应的结果列表，结构为 results[i][j] = 第i个启发函数在第j个状态下的结果
    """

    results = [[] for _ in range(len(heuristics))]  # 为每个启发式函数创建一个结果列表

    for initial_state in initial_states:
        print('Solving state')
        visualize_state(initial_state)  # 可视化当前拼图状态（需在 Jupyter 中运行）

        # 构造问题实例
        problem = Taquin(initial_state, goal_state, len(initial_state[0]))

        # 对每个启发函数执行 A* 搜索
        for k in range(len(heuristics)):
            heuristic = heuristics[k]
            print('Running A* with', heuristic.__name__)  # 打印当前使用的启发函数名

            start_time = time.time()  # 记录开始时间
            solution_node, num_explorations = a_etoile(problem, heuristic, max_it)  # 运行 A*
            end_time = time.time()  # 记录结束时间

            execution_time = end_time - start_time  # 计算执行时间

            if solution_node:
                path_length = len(solution_node.solution())  # 计算解决方案路径长度
            else:
                path_length = -1  # 如果未找到解，路径长度设为 -1

            # 将结果记录到对应启发函数的结果列表中
            results[k].append({
                "initial_state": initial_state,
                "algorithm": heuristic.__name__,     # 启发函数名
                "num_explorations": num_explorations,  # 节点扩展次数
                "path_length": path_length,          # 解路径长度
                "execution_time": execution_time     # 执行时间（秒）
            })

    return results  # 返回所有启发函数的结果




In [ ]:
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]  # 目标状态
heuristics = [misplaced_tiles, manhattan_distance]  # 两种启发式函数

# 执行比较函数
results = compare_algorithms(initial_states, goal_state, heuristics, max_it=20000)

# 将结果分别提取出来（便于后续可视化或表格展示）
results_misplace = results[0]
results_manhattan = results[1]

print('✅ Finished !!')

Solving state


1,2,3
4,5,
6,7,8


Running A* with misplaced_tiles
Running A* with manhattan_distance
Solving state


1,2,3
,5,6
4,7,8


Running A* with misplaced_tiles
Running A* with manhattan_distance
Solving state


...

Running A* with misplaced_tiles
Running A* with manhattan_distance
Solving state


1,2,3
4,6,
7,5,8


Running A* with misplaced_tiles
Running A* with manhattan_distance
Solving state


1,3,6
4,2,5
7,,8


Running A* with misplaced_tiles
Running A* with manhattan_distance


#### **Utilisez le code ci-dessous pour visualiser vos résultats et commentez**
请使用下面的代码来可视化你的结果，并进行评论。

In [ ]:
import pandas as pd              # 用于构建和处理表格数据
import numpy as np              # 常用数值计算库（本例中未使用）
from tabulate import tabulate   # 将 pandas 表格漂亮地打印为文本表格（需先 pip install tabulate）

# === 构建表格数据 ===
Data = []

# 遍历所有的初始状态（共15个 Taquin 拼图实例）
for i in range(len(initial_states)):
    # 将每个算法在当前状态下的结果提取出来组成一行
    Data.append([
        int(i+1),  # 状态编号（从1开始）

        # 扩展节点数（越少越优）
        results_BFS[i]['num_explorations'],
        results_DFS[i]['num_explorations'],
        results_misplace[i]['num_explorations'],
        results_manhattan[i]['num_explorations'],

        # 解路径长度（步数，越短越优；-1表示未找到）
        results_BFS[i]['path_length'],
        results_DFS[i]['path_length'],
        results_misplace[i]['path_length'],
        results_manhattan[i]['path_length'],

        # 执行时间（秒，越小越优，四舍五入到小数点后2位）
        round(results_BFS[i]['execution_time'], 2),
        round(results_DFS[i]['execution_time'], 2),
        round(results_misplace[i]['execution_time'], 2),
        round(results_manhattan[i]['execution_time'], 2)
    ])

# 使用 Pandas 创建 DataFrame 表格
df = pd.DataFrame(Data, columns=[
    "State",                 # 状态编号
    "Nodes explored BFS",   # BFS 扩展的节点数
    "Nodes explored DFS",   # DFS 扩展的节点数
    "Nodes explored A*H1",  # A* + 启发函数 H1（错位块）
    "Nodes explored A*H2",  # A* + 启发函数 H2（曼哈顿距离）

    "Path length BFS",      # BFS 路径长度
    "Path length DFS",      # DFS 路径长度
    "Path length A*H1",     # A* + H1 路径长度
    "Path length A*H2",     # A* + H2 路径长度

    "Execution Time BFS",   # BFS 执行时间（秒）
    "Execution Time DFS",   # DFS 执行时间（秒）
    "Execution Time A*H1",  # A* + H1 执行时间
    "Execution Time A*H2"   # A* + H2 执行时间
])

# 使用 tabulate 美化输出表格（你也可以直接 print(df)）
print(tabulate(df, headers='keys', tablefmt='pretty'))
